In [ ]:
import numpy as np

import sys
sys.path.append(f'../src')
from importlib import reload 
import numpy as np
import torch
from torch.optim import Adam
import gym
import time

import sac_tri
import sac_tri_envs
import core_tri

reload(sac_tri);
reload(sac_tri_envs);
reload(core_tri);

## Train a two-level system in the thermalization dominated regime

In [ ]:
a_val = 0.7

env_params = {
    "g": 1.,                      
    "b": 1.,
    "min_u": -0.8,
    "max_u": 0.8,
    "e0": 5.,                     
    "dt": 0.07,   
    "a": a_val,
    "pow_coeff": 1.,
    "diss_coeff": 1.,
}  
training_hyperparams = {
    "BATCH_SIZE": 256,              #batch size
    "LR": 0.001,                    #learning rate
    "ALPHA_LR": 0.003,
    "H_D_START": np.log(3.),        #the exploration coeff
    "H_D_END": 0.01,                #the exploration coeff
    "H_D_DECAY": 30000,            #the exploration coeff, in "units" of steps
    "H_C_START": 0.8,              #the exploration coeff
    "H_C_END": -3.,                #3.5 INSTEAD OF -7 SINCE HERE 1 ACTION, WHILE -7 2 ACTIONS
    "H_C_DECAY": 60000,           #the exploration coeff, in "units" of steps
    "REPLAY_MEMORY_SIZE": 80000,  #IT WAS 2000
    "POLYAK": 0.995,                #polyak coefficient
    "LOG_STEPS": 1000,              #save logs and display training every number of steps
    "GAMMA": 0.998,                 #RL discount factor
    "HIDDEN_SIZES": (256,128),      #size of hidden layers 
    "SAVE_STATE_STEPS": 160000,      #saves complete state of trainig every number of steps
    "INITIAL_RANDOM_STEPS": 5000,   #number of initial uniformly random steps
    "UPDATE_AFTER": 1000,           #start minimizing loss function after initial steps
    "UPDATE_EVERY": 50,             #performs this many updates every this many steps
    "USE_CUDA": False,               #use cuda for computation
    "MIN_COV_EIGEN": 1.e-8,
    "DONT_SAVE_MEMORY": False
}
log_info = {
    "log_running_reward": True,     #log running reward 
    "log_running_loss": True,       #log running loss
    "log_actions": True,            #log chosen actions
    "extra_str": f"_a={a_val}_newenv" #extra string to append to training folder
}

train = sac_tri.SacTrain()
train.initialize_new_train(sac_tri_envs.TwoLevelBosonicFeedbackDemonPowDiss, env_params, training_hyperparams, log_info)

In [ ]:
train.train(160000)

## Evaluate the current policy

In [ ]:
train.evaluate_current_policy(deterministic=True, steps=10000,
                              actions_to_plot=200, gamma=0.9999)

## Save the state

In [ ]:
train.save_full_state()

## Load the state

In [ ]:
state_dir = "../data/2022_07_05-14_32_42_a=0.95"

train = sac_tri.SacTrain()
train.load_train(state_dir,no_train=False)

### Continue training

In [ ]:
train.train(50)

## view training status

In [ ]:
train.plot_logs()